## Demonstrate the use of potential_cloud_first_pass_ee for Sentinel 2 data

You need to install [geemap](https://github.com/giswqs/geemap) to visualize the outputs.

In [1]:
import ee
ee.Initialize()

In [2]:
# Select a cloudy Sentinel2 image
date_str='2017-04-15' 
lonlat=[38.530867, 30.236383] 
date = ee.Date(date_str)
geom = ee.Geometry.Point(lonlat[0], lonlat[1]) # lon/lat

S2_img = ee.Image(
  ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(geom)
    .filterDate(date,date.advance(3,'month'))
    .sort('system:time_start')
    .first()
  )

In [3]:
# Instantiate the fmaskconfig object
cmdargs = type('',(),{})()
cmdargs.mincloudsize=300
cmdargs.cloudprobthreshold = 35
cmdargs.nirsnowthreshold=0.4
cmdargs.greensnowthreshold=0.35
cmdargs.parallaxtest=True 
cmdargs.cloudbufferdistance = 150 # Default (see sentinel2Stacked.py)
cmdargs.shadowbufferdistance = 300 # Default
toaImgInfo = type('',(),{})()
toaImgInfo.xRes = 10
from fmask import config
fmaskConfig = config.FmaskConfig(config.FMASK_SENTINEL2) # default Sentinel2 configuration options
fmaskConfig.setTOARefScaling(10000.0) 
fmaskConfig.setMinCloudSize(cmdargs.mincloudsize)
fmaskConfig.setEqn17CloudProbThresh(cmdargs.cloudprobthreshold / 100)    
fmaskConfig.setEqn20NirSnowThresh(cmdargs.nirsnowthreshold)
fmaskConfig.setEqn20GreenSnowThresh(cmdargs.greensnowthreshold)
fmaskConfig.setCloudBufferSize(int(cmdargs.cloudbufferdistance / toaImgInfo.xRes))  
fmaskConfig.setShadowBufferSize(int(cmdargs.shadowbufferdistance / toaImgInfo.xRes))
fmaskConfig.sen2displacementTest = True 

# added by this module:
other_args = type('', (), {})()
other_args.N_focal_max = 5
other_args.res = 10
other_args.replace_and = False

In [4]:
#ref = S2_img.divide(fmaskConfig.TOARefScaling)
# Call potentialCloudFirstPass_ee to get the pcp first pass output:
from fmask.sen2FmaskEE import potentialCloudFirstPass_ee 
pcp, waterTest, clearLand, variabilityProb, snowmask = potentialCloudFirstPass_ee(fmaskConfig, S2_img, other_args)

In [6]:
# Visualize the returned layers:
index_viz = {'min':0, 'max':1}
mask_viz = {'palette':["#ffffff","#d7191c"]}
cdi_viz = {'min': -1, 'max': 1}
import geemap
Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=10)
Map.addLayer(S2_img.divide(fmaskConfig.TOARefScaling), {'bands':['B4','B3','B2'],'min':0, 'max':0.25}, 'Sentinel 2 image')
Map.addLayer(waterTest.selfMask(), mask_viz, 'Water test')
Map.addLayer(clearLand.selfMask(), mask_viz, 'Clear-sky land')
Map.addLayer(variabilityProb, index_viz, 'Variability prob')
Map.addLayer(snowmask.selfMask(), mask_viz, 'Snow mask')
Map.addLayer(pcp.selfMask(), mask_viz, 'PCP')
Map

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…